In [1]:
import pandas as pd
import geopandas as gpd
from p_tqdm import p_map,p_umap,p_uimap,p_imap


import glob, re , os 
import numpy as np
dloc = '2021-oa-data'
oloc = '../static/'
tloc = 'temp/'

In [2]:
# geom =  'Output_Areas_(Dec_2021)_Boundaries_Full_Clipped_EW_(BFC).geojson').set_index('OA21CD')gpd.read_file(

# geom.to_file('geom.shp') 
geom = gpd.read_file('geom.shp').set_index('OA21CD')
geom = geom.geometry

In [3]:
import random
from shapely.geometry import Point

def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        cds = [random.uniform(minx, maxx), random.uniform(miny, maxy)]
        pnt = Point(cds)
        if polygon.contains(pnt):
            points.append(cds)
    return points

In [4]:
# typen = 'TS017_hh_size_5a_2021'

# typen =  'TS021_ethnic_group_tb_6a_2021'
typen = 'TS002_legal_partnership_status_3a_2021'

data = pd.read_csv(dloc+'/'+typen+'.csv').set_index('Geography code')

In [6]:
start = list(data.columns).index('Classification') + 1
end = list(data.columns).index('Total')
selection = list(data.columns)[start:end]
selection

data = data[selection]

In [5]:
def makepoints(area_chunks):
    '''
    This functions processes the points for several areas'''
    dummy = []
    
    # if type(area_chunks) != 'list': area_chunks = [area_chunks]

    for area in area_chunks:
        row = data.loc[area]

        for cid,col in enumerate(selection):

            template = '{ "type": "Feature", "properties": { "cat": ' + str(cid)+' }, "geometry": { "type": "Point", "coordinates": [ %0.6f, %0.6f ] } },'

            #need a format we can turn into geojson
            dummy.extend([template%(p1,p2) for p1,p2 in generate_random(int(row[col]), geom.loc[area])])


    np.random.shuffle(dummy)
    return '\n'.join(dummy)


In [7]:
# london = open('south.txt').read().split(',')
oas = list(set(data.index) & set(geom.index))
len(oas), len(data), len(oas)/len(data), len(oas)/15000

(188880, 232183, 0.813496250802169, 12.592)

In [8]:
f = open(tloc + typen + '.geojson','w')

f.write('''{
"type": "FeatureCollection",
"features": [
''')

nit = 15000
ncores = 8
fstr = ''
counter = 0 

split = np.array_split(oas,nit)

# ratio optimised for macbook pro 8c 32gb
for res in p_uimap(makepoints,split,num_cpus=ncores):
    # allpoints.extend(res)

    fstr += res
    counter += 1

    if counter%20:
        f.write(fstr)
        fstr=''

f.write(fstr[:-3]+ '\n]}')
f.close()   

100%|██████████| 15000/15000 [16:06<00:00, 15.52it/s]  


In [9]:
# use parallel! as newline delimited geojson
'-z16 -Bg'
os.system(f'rm -rf {oloc}/{typen}')
os.system(f'/usr/local/bin/tippecanoe -f -l {typen} -r5 --drop-fraction-as-needed --no-tile-compression --no-tile-size-limit --no-feature-limit -P --output-to-directory="{oloc}{typen}" -zg -B13 -Z8 {tloc}*')



temp/TS002_legal_partnership_status_3a_2021.geojson:3034458: Found ] not in an array
In JSON object {"type":"Feature","properties":{"cat":1},"geometry":{"type":"Point","coordinates":[-4.159411,50.417064]}}
temp/TS002_legal_partnership_status_3a_2021.geojson:3034553: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
temp/TS017_hh_size_5a_2021.geojson:1547976: Found ] not in an array
In JSON object {"type":"Feature","properties":{"cat":1},"geometry":{"type":"Point","coordinates":[-1.98423,52.612269]}}
temp/TS017_hh_size_5a_2021.geojson:1547967: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
temp/TS021_ethnic_group_tb_6a_2021.geojson:3723817: Found ] not in an array
In JSON object {"type":"Feature","properties":{"cat":3},"geometry":{"type":"Point","coordinates":[0.124095,51.534984]}}
temp/TS021_ethnic_group_tb_6a_2021.geojson:3723910: Reached EOF without all containers being 

0

In [16]:
typen

'TS002_legal_partnership_status_3a_2021'

In [10]:
st='' 
key = '`'
for i,s in enumerate(selection):
    st+= f'"{s}" , colour[{i}],'
    key+="<span style='color:${colour[%d]}'> %s </span><br>"%(i,s)
st+='"#ccc"'
key+='`'

print(st)
print()
print(key)


"Married or in a registered civil partnership" , colour[0],"Other marital or civil partnership status" , colour[1],"#ccc"

`<span style='color:${colour[0]}'> Married or in a registered civil partnership </span><br><span style='color:${colour[1]}'> Other marital or civil partnership status </span><br>`


In [11]:
selection

['Married or in a registered civil partnership',
 'Other marital or civil partnership status']

In [12]:
sm = data.sum()

nsm = sm/sm.max()

nsm.values * 100

array([ 81.21719323, 100.        ])